In [ ]:
import os  
import sys 
sys.path.insert(0, "/home/jifengwang/data/25_polymer_database/AI_prediction/GAT")
sys.path.insert(0, "/home/jifengwang/data/25_polymer_database/AI_prediction/GCN")
sys.path.insert(0, "/home/jifengwang/data/25_polymer_database/AI_prediction/MLP")
sys.path.insert(0, "/home/jifengwang/data/25_polymer_database/AI_prediction/PolyBERT")

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost as xgb
import csv as csv
from xgboost import plot_importance
from matplotlib import pyplot
from sklearn.model_selection import cross_val_score,KFold
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV   #Performing grid search
from scipy.stats import skew
from collections import OrderedDict
from sklearn.inspection import permutation_importance
import shap
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold, KFold
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
import joblib
import numbers
import torch
from torch_geometric.loader import DataLoader  

In [ ]:
# plt.style.use("~/style.mplstyle")

In [4]:
# 固定随机种子
def set_seed(seed):
    import random
    random.seed(seed)  # 固定 Python 的随机种子
    np.random.seed(seed)  # 固定 NumPy 的随机种子
    torch.manual_seed(seed)  # 固定 PyTorch 的随机种子
    torch.cuda.manual_seed(seed)  # 固定 GPU 上的随机种子
    torch.backends.cudnn.deterministic = True  # 确保 CuDNN 的确定性
    torch.backends.cudnn.benchmark = False  # 禁用非确定性算法优化

In [5]:
# 使用 joblib 保存模型
def save_model_with_joblib(model, filename):
    """
    使用 joblib 保存XGBoost模型。

    参数:
    model -- 训练好的机器学习模型
    filename -- 模型保存的文件名，应以 .joblib 结尾

    返回:
    None
    """
    joblib.dump(model, filename)

In [6]:
import torch
# 加载数据集
def load_data_list(file_path):
    """
    从指定文件路径加载数据列表。

    参数：
    file_path (str): 数据列表文件的路径。

    返回：
    list: 加载的数据列表。
    """
    return torch.load(file_path)

def data_loader(property, N=None):
    if not N:
        property_load_file_path = f'experiment_dataset/{property}_data_list.pt' # 总数据集
        property_load_train_dataset_path = f'experiment_dataset/train_{property}_data_list.pt' # 训练集
        property_load_val_dataset_path = f'experiment_dataset/val_{property}_data_list.pt' # 验证集
        property_load_test_dataset_path = f'experiment_dataset/test_{property}_data_list.pt' # 测试集
    else:
        property_load_file_path = f'experiment_dataset_N={N}/{property}_data_list.pt' # 总数据集
        property_load_train_dataset_path = f'experiment_dataset_N={N}/train_{property}_data_list.pt' # 训练集
        property_load_val_dataset_path = f'experiment_dataset_N={N}/val_{property}_data_list.pt' # 验证集
        property_load_test_dataset_path = f'experiment_dataset_N={N}/test_{property}_data_list.pt' # 测试集

    # 加载数据集
    property_loaded_data_list = load_data_list(property_load_file_path)
    property_loaded_train_data_list = load_data_list(property_load_train_dataset_path)
    property_loaded_val_data_list = load_data_list(property_load_val_dataset_path)
    property_loaded_test_data_list = load_data_list(property_load_test_dataset_path)

    return property_loaded_train_data_list, property_loaded_val_data_list, property_loaded_test_data_list

In [7]:
import xgboost as xgb
import optuna
import time
import torch
import numpy as np
from sklearn.metrics import mean_squared_error

def extract_features_targets(data_list):
    """
    从数据列表中提取moldescriptor作为特征X和y中指定索引的值作为目标Y。
    
    Args:
        data_list (list): 包含Data对象的列表。
        feature (str): 可选"2Ddescriptors"或“fp”，代表训练模型时使用的特征
        
    Returns:
        X (np.ndarray): 特征矩阵。
        Y (np.ndarray): 目标向量。
    """
    X = []
    Y = []
    for data in data_list:
        fp = data.morgan_fp.numpy().flatten() # 采用分子描述符作为特征
        X.append(fp)
        # 提取目标y值
        target = data.y.numpy().flatten()
        Y.append(target)
        
    # print(X)
    
    X = np.array(X)
    Y = np.array(Y)
    return X, Y

def XGBoost_model_optuna(train_list, val_list):
    """
    使用Optuna优化XGBoost模型参数，并训练和评估模型。
    
    Args:
        train_list (list): 训练集列表（包含所有阳离子和阴离子数据）。
        val_list (list): 验证集列表（包含所有阳离子和阴离子数据）。

    Returns:
        best_model (xgb.XGBRegressor): 训练好的XGBoost模型。
    """
    start_time = time.time()
    
    # 提取训练集特征和目标
    X_train, y_train = extract_features_targets(train_list)
    
    # 提取验证集特征和目标
    X_val, y_val = extract_features_targets(val_list)
    
    def objective(trial):
        # 定义搜索空间
        param = {
            "max_depth": trial.suggest_int("max_depth", 3, 10),  # 决策树最大深度
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),  # 学习率
            "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=50),  # 树的数量
            "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),  # 最小叶子节点样本权重和
            "gamma": trial.suggest_float("gamma", 0, 5),  # 分裂所需的最小损失减少值
            "subsample": trial.suggest_float("subsample", 0.6, 1.0),  # 数据采样比例
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),  # 特征采样比例
            "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10, log=True),  # L1正则化
            "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10, log=True),  # L2正则化
        }
        
        # 初始化模型
        model = xgb.XGBRegressor(
            objective="reg:squarederror",
            **param,
            random_state=42,
            n_jobs=-1
        )
        
        # 训练模型
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            verbose=False
        )
        
        # 验证集预测
        y_pred = model.predict(X_val)
        
        # 返回均方误差
        return mean_squared_error(y_val, y_pred)
    
    # 使用Optuna进行参数优化
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=100, timeout=600)  # X次试验，最多优化600秒
    
    # 获取最佳参数
    best_params = study.best_params
    #print("最佳参数: ", best_params)
    
    # 使用最佳参数训练最终模型
    best_model = xgb.XGBRegressor(
        objective="reg:squarederror",
        **best_params,
        random_state=42,
        n_jobs=-1
    )
    best_model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=False
    )
    
    # 输出训练时间
    end_time = time.time()
    #print("训练时间: {:.2f} 秒".format(end_time - start_time))
    
    # 评估最终模型
    y_final_pred = best_model.predict(X_val)
    final_mse = mean_squared_error(y_val, y_final_pred)
    print("最终验证集均方误差 (MSE): {:.4f}".format(final_mse))
    
    return best_model

In [8]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

def evaluate_model(model, val_list):
    """
    对模型在验证集上进行评估，并打印出不同的性能指标。
    
    参数:
        model -- 已训练的机器学习模型
        val_list -- 验证集列表（包含所有数据）
        figname (str) -- 保存图像的文件名，不包括扩展名，默认为 "model_evaluation"
        figsize (tuple) -- 图像的大小，默认为 (6, 6)
        output (bool) -- 是否保存图像，默认为 False
    
    返回:
        None
    """
    
    # 提取验证集特征和目标
    X_test, y_test = extract_features_targets(val_list)
    
    # 设置字体为 Arial
    plt.rcParams["font.family"] = "Arial"
    
    # 使用模型对测试集进行预测
    y_pred = model.predict(X_test)
    
    # 计算MSE
    mse = mean_squared_error(y_test, y_pred)
    print(f"MSE (Mean Squared Error): {mse:.2f}")
    
    # 计算MAE
    mae = mean_absolute_error(y_test, y_pred)
    print(f"MAE (Mean Absolute Error): {mae:.2f}")
    
    # 计算RMSE
    rmse = np.sqrt(mse)  # RMSE是MSE的平方根
    print(f"RMSE (Root Mean Squared Error): {rmse:.2f}")
    
    # 计算R^2 Score
    r2 = r2_score(y_test, y_pred)
    print(f"R^2 Score: {r2:.2f}")
   
    return mae, mse, r2

In [12]:
def batch_train_xgb_model(target_property_col_list, N=None, seed=42):
    """
    针对每个 (property, model) 组合训练并评估，返回测试集指标汇总 DataFrame。

    参数：
        df (pd.DataFrame): 原始数据（若需要，可在函数内加载）。
        target_property_col_list (List[str]): 需要预测的性质名称列表。
        model_type_list (List[str]): 模型类型列表，如 ["GAT","GCN","MLP","PolyBERT"]。
        epochs (int): 训练轮数。
        seed (int): 随机种子。

    返回:
        pd.DataFrame: 包含列 [property, model_type, test_mse, test_mae, test_r2]。
    """
    
    # 设置随机种子
    set_seed(seed)
    
    # 用于汇总所有结果的列表
    results = []

    for property in target_property_col_list:
        print(f"→ Training XGBoost on property: {property}")
        try:
            # 加载数据
            train_list, val_list, test_list = data_loader(property)
            
            # XgBoost模型
            model = XGBoost_model_optuna(
                train_list=train_list,
                val_list=val_list
            )

            property_test_mae, property_test_mse, property_test_r2 = evaluate_model(model, test_list)

            if not N:
                save_path = f'model/{property}_xgb_model.joblib'
            else:
                save_path = f'model_N={N}/{property}_xgb_model.joblib'
                
            save_model_with_joblib(model, save_path)
        except Exception as e:
            # 出错时跳过，并把指标置为 NaN
            print(f"⚠️  XGBoost failed on {property}: {e}")
            property_test_mae, property_test_mse, property_test_r2 = np.nan
        
        # 将结果追加到列表
        results.append({
            "property": property,
            "model_type": "XGBoost",
            "test_mse": property_test_mse,
            "test_mae": property_test_mae,
            "test_r2": property_test_r2
        })

    # 构造并返回 DataFrame
    return pd.DataFrame(results)


In [13]:
target_property_col_list = ['Bandgap', 'CO2_Permeability',
       'Compressive_Strength', 'Crystallization_Temperature',
       'Elongation_at_Break', 'Flexural_Strength', 'Tg', 'H2_Permeability',
       'Hardness', 'Impact_Strength', 'Ion_Exchange_Capacity',
       'Limiting_Oxygen_Index', 'Lower_Critical_Solution_Temperature', 'Tm',
       'Methanol_Permeability', 'O2_Permeability', 'Refractive_Index',
       'Swelling_Degree', 'Thermal_Conductivity', 'Tensile_Strength', 'Td',
       'Upper_Critical_Solution_Temperature', 'Water_Contact_Angle',
       'Water_Uptake', 'Youngs_Modulus','Dielectric_Constant_Total'
       ]

In [14]:
df_xgb_test_performance_summary = batch_train_xgb_model(target_property_col_list, seed=42)

/tmp/ipykernel_2801521/2484298480.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file_path)
[I 2025-05-26 13:15:57,926] A new study created in memory

→ Training XGBoost on property: Bandgap


[I 2025-05-26 13:15:58,295] Trial 0 finished with value: 0.7351091504096985 and parameters: {'max_depth': 7, 'learning_rate': 0.09487763203593454, 'n_estimators': 600, 'min_child_weight': 3, 'gamma': 0.6754833469225335, 'subsample': 0.7284300744932557, 'colsample_bytree': 0.8136151415027733, 'reg_alpha': 0.009757073400680369, 'reg_lambda': 2.78668783228282e-07}. Best is trial 0 with value: 0.7351091504096985.
[I 2025-05-26 13:15:58,714] Trial 1 finished with value: 0.7342790961265564 and parameters: {'max_depth': 9, 'learning_rate': 0.06412596369579308, 'n_estimators': 950, 'min_child_weight': 10, 'gamma': 4.8754536166543545, 'subsample': 0.97886527205672, 'colsample_bytree': 0.8910865999847878, 'reg_alpha': 1.493063524602324e-08, 'reg_lambda': 2.488911203142071e-05}. Best is trial 1 with value: 0.7342790961265564.
[I 2025-05-26 13:15:59,096] Trial 2 finished with value: 0.8037037253379822 and parameters: {'max_depth': 8, 'learning_rate': 0.045007115614645925, 'n_estimators': 800, 'min

最终验证集均方误差 (MSE): 0.6481
MSE (Mean Squared Error): 0.99
MAE (Mean Absolute Error): 0.75
RMSE (Root Mean Squared Error): 0.99
R^2 Score: -0.09
→ Training XGBoost on property: CO2_Permeability


[I 2025-05-26 13:16:20,338] Trial 0 finished with value: 166077.921875 and parameters: {'max_depth': 8, 'learning_rate': 0.06934399020379584, 'n_estimators': 600, 'min_child_weight': 8, 'gamma': 2.6202744748693108, 'subsample': 0.7913865967216904, 'colsample_bytree': 0.864159205055794, 'reg_alpha': 0.00010157910122559491, 'reg_lambda': 0.00551456883001129}. Best is trial 0 with value: 166077.921875.
[I 2025-05-26 13:16:20,962] Trial 1 finished with value: 134162.203125 and parameters: {'max_depth': 4, 'learning_rate': 0.0945769996138179, 'n_estimators': 700, 'min_child_weight': 3, 'gamma': 3.801830586738308, 'subsample': 0.9871231254880446, 'colsample_bytree': 0.8081098342556974, 'reg_alpha': 4.937296236288645, 'reg_lambda': 6.205860281578553}. Best is trial 1 with value: 134162.203125.
[I 2025-05-26 13:16:21,515] Trial 2 finished with value: 179841.21875 and parameters: {'max_depth': 6, 'learning_rate': 0.0776149416355624, 'n_estimators': 450, 'min_child_weight': 4, 'gamma': 0.9152560

...

最终验证集均方误差 (MSE): 518.3847
MSE (Mean Squared Error): 798.12
MAE (Mean Absolute Error): 22.09
RMSE (Root Mean Squared Error): 28.25
R^2 Score: 0.19
→ Training XGBoost on property: Youngs_Modulus


[I 2025-05-26 13:40:20,412] Trial 0 finished with value: 186077376.0 and parameters: {'max_depth': 4, 'learning_rate': 0.13421914085634276, 'n_estimators': 800, 'min_child_weight': 9, 'gamma': 1.8882182400409069, 'subsample': 0.624748753123236, 'colsample_bytree': 0.9395944983124257, 'reg_alpha': 0.00018768713334449812, 'reg_lambda': 5.650443830577374e-07}. Best is trial 0 with value: 186077376.0.
[I 2025-05-26 13:40:20,920] Trial 1 finished with value: 2633116.25 and parameters: {'max_depth': 8, 'learning_rate': 0.021144250293490423, 'n_estimators': 300, 'min_child_weight': 5, 'gamma': 3.2707207444212676, 'subsample': 0.7544729920445318, 'colsample_bytree': 0.8647617280479636, 'reg_alpha': 1.1812984596603833, 'reg_lambda': 6.650586141201791e-08}. Best is trial 1 with value: 2633116.25.
[I 2025-05-26 13:40:21,570] Trial 2 finished with value: 1960387.25 and parameters: {'max_depth': 5, 'learning_rate': 0.12095095141495672, 'n_estimators': 550, 'min_child_weight': 4, 'gamma': 1.03468807

最终验证集均方误差 (MSE): 1062607.7500
MSE (Mean Squared Error): 243333856.00
MAE (Mean Absolute Error): 7541.97
RMSE (Root Mean Squared Error): 15599.16
R^2 Score: -0.07
→ Training XGBoost on property: Dielectric_Constant_Total


[I 2025-05-26 13:41:44,578] Trial 0 finished with value: 0.45438316464424133 and parameters: {'max_depth': 3, 'learning_rate': 0.03172313835888137, 'n_estimators': 300, 'min_child_weight': 3, 'gamma': 3.791361893133206, 'subsample': 0.6356501020046038, 'colsample_bytree': 0.986232990846664, 'reg_alpha': 0.13361411086637956, 'reg_lambda': 8.739697323927404}. Best is trial 0 with value: 0.45438316464424133.
[I 2025-05-26 13:41:45,072] Trial 1 finished with value: 0.40861040353775024 and parameters: {'max_depth': 3, 'learning_rate': 0.14402694558860274, 'n_estimators': 700, 'min_child_weight': 1, 'gamma': 4.4200430165789495, 'subsample': 0.7619484609535055, 'colsample_bytree': 0.9323305263627488, 'reg_alpha': 0.04667425677798405, 'reg_lambda': 3.153732247711427e-07}. Best is trial 1 with value: 0.40861040353775024.
[I 2025-05-26 13:41:45,464] Trial 2 finished with value: 0.4238104522228241 and parameters: {'max_depth': 7, 'learning_rate': 0.019220763245937945, 'n_estimators': 400, 'min_ch

最终验证集均方误差 (MSE): 0.3380
MSE (Mean Squared Error): 0.29
MAE (Mean Absolute Error): 0.37
RMSE (Root Mean Squared Error): 0.54
R^2 Score: 0.50


In [15]:
df_xgb_test_performance_summary

,property,model_type,test_mse,test_mae,test_r2
0,Bandgap,XGBoost,9.889547e-01,0.746056,-0.085991
1,CO2_Permeability,XGBoost,3.849748e+06,1039.618774,0.081336
2,Compressive_Strength,XGBoost,2.842819e+05,206.321548,-0.070669
3,Crystallization_Temperature,XGBoost,3.984959e+03,39.250500,-0.002317
4,Elongation_at_Break,XGBoost,9.082938e+04,241.451294,-1.479438
5,Flexural_Strength,XGBoost,5.824814e+05,287.703308,-0.133217
6,Tg,XGBoost,2.209741e+03,22.674313,0.734598
7,H2_Permeability,XGBoost,1.516590e+07,1894.599731,-0.142308
8,Hardness,XGBoost,1.427279e+06,768.249084,-0.094963
9,Impact_Strength,XGBoost,7.631493e+03,43.790668,-0.155186


In [ ]:
df_xgb_test_performance_summary.to_csv("xgb_model_test_performance_summary.csv", index=None)